In [102]:
import requests
import pandas as pd
import numpy as np
import datetime
import os
import zipfile

In [85]:
pip install --upgrade pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 4.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.0
    Uninstalling pandas-2.0.0:
      Successfully uninstalled pandas-2.0.0
--- Logging error ---
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pip/_internal/utils/logging.py", 

In [103]:
#List of countries and regions are here: https://unstats.un.org/unsd/methodology/m49/overview/ but website using
#javascript and not providing link for csv file directly
#Workaround:
#Using a github page to download a csv file
url = "https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/raw/master/all/all.csv"
filename = "countries.csv"

response = requests.get(url)

if response.status_code == 200:
    with open(filename, "wb") as file:
        file.write(response.content)
    print("CSV file downloaded successfully.")
else:
    print("Failed to download the CSV file.")

CSV file downloaded successfully.


In [104]:
#To find a file of your interest: https://data.worldbank.org/indicator
#Clicking on a subset allows to get address to download csv-file

#These are addresses to obtain the data for GDP per capita, life expectancy and literacy rate
url_list=["https://api.worldbank.org/v2/en/indicator/NY.GDP.PCAP.PP.CD?downloadformat=csv", \
          "https://api.worldbank.org/v2/en/indicator/SP.DYN.LE00.IN?downloadformat=csv",\
            "https://api.worldbank.org/v2/en/indicator/SE.ADT.LITR.ZS?downloadformat=csv"]

In [105]:
#Also, giving names for the data to be obtained
data_list=["gdp_per_capita", "life_expectancy", "literacy"]

In [106]:
#First, making folder to download the data from web, names based on data_list
for d in data_list:
    if not os.path.exists(d):
        os.makedirs(d)
    if not os.path.exists(d+"_unpacked"):
        os.makedirs(d+"_unpacked")

In [107]:
#Here one by one, I dowload the data and put into a right folder
for d, u in zip(data_list, url_list):
    response = requests.get(u)
    if response.status_code == 200:
        with open(d+"/"+d+".zip", "wb") as file:
            file.write(response.content)
        print("Zip file downloaded successfully!")
    else:
        print("Failed to retrieve the zip file.")

Zip file downloaded successfully!
Zip file downloaded successfully!
Zip file downloaded successfully!


In [108]:
#Unpacking the data as well
for d in data_list:
    zip_file_path=d+"/"+d+".zip"
    destination_folder=d+"_unpacked"
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(destination_folder)
    print("Extraction completed!")


Extraction completed!
Extraction completed!
Extraction completed!


In [109]:
#To remove excess rows in the csv file, an additional method:
def copy_file(source_file, destination_file):
    with open(source_file, 'r') as source:
        with open(destination_file, 'w') as destination:
            counter=0
            for line in source:
                if(counter>3):
                    destination.write(line)
                counter=counter+1

In [9]:
#Removing extra rows from the csv file:

for d in data_list:
    file_list=os.listdir(d+"_unpacked")
    file_list.sort()
    source_file=d+"_unpacked/"+file_list[0]
    destination_file=d+".csv"
    copy_file(source_file, destination_file)

In [110]:
#Using pandas, going to create the dataframe list:
df_list=[]
for d in data_list:
    temp_df=pd.read_csv(d+".csv")
    df_list.append(temp_df)

In [111]:
#The last column is empty, must be removed
df_list[0].head(5)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Aruba,ABW,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,36846.848285,37343.912963,37583.840323,38865.188195,41679.238190,42501.641554,34971.009913,42698.359872,NaN,NaN
1,Africa Eastern and Southern,AFE,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3470.550675,3498.125396,3591.099809,3635.564622,3724.868177,3777.972368,3621.058106,3839.470058,NaN,NaN
2,Afghanistan,AFG,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2110.829568,2128.125938,2023.834656,2096.093111,2109.929296,2167.704111,2076.138380,1665.805842,NaN,NaN
3,Africa Western and Central,AFW,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,4143.177054,4075.994459,3998.951876,4045.303263,4160.540965,4264.731035,4174.504565,4409.450961,NaN,NaN
4,Angola,AGO,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,8123.048065,7274.090475,7027.146634,7216.061373,7042.923829,6881.076241,6362.636076,6491.125578,NaN,NaN


In [112]:
#Running the deletion
for n in range(len(df_list)):
    df_list[n]=df_list[n][df_list[n].columns[:-1]]

In [118]:
#Checking
df_list[0].head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,ABW,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,36932.184164,36846.848285,37343.912963,37583.840323,38865.188195,41679.238190,42501.641554,34971.009913,42698.359872,NaN
1,Africa Eastern and Southern,AFE,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3333.520911,3470.550675,3498.125396,3591.099809,3635.564622,3724.868177,3777.972368,3621.058106,3839.470058,NaN
2,Afghanistan,AFG,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2062.059176,2110.829568,2128.125938,2023.834656,2096.093111,2109.929296,2167.704111,2076.138380,1665.805842,NaN
3,Africa Western and Central,AFW,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3950.948889,4143.177054,4075.994459,3998.951876,4045.303263,4160.540965,4264.731035,4174.504565,4409.450961,NaN
4,Angola,AGO,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,7643.921944,8123.048065,7274.090475,7027.146634,7216.061373,7042.923829,6881.076241,6362.636076,6491.125578,NaN


In [114]:
print(df_list)

[                    Country Name Country Code   
0                          Aruba          ABW  \
1    Africa Eastern and Southern          AFE   
2                    Afghanistan          AFG   
3     Africa Western and Central          AFW   
4                         Angola          AGO   
..                           ...          ...   
261                       Kosovo          XKX   
262                  Yemen, Rep.          YEM   
263                 South Africa          ZAF   
264                       Zambia          ZMB   
265                     Zimbabwe          ZWE   

                                    Indicator Name     Indicator Code  1960   
0    GDP per capita, PPP (current international $)  NY.GDP.PCAP.PP.CD   NaN  \
1    GDP per capita, PPP (current international $)  NY.GDP.PCAP.PP.CD   NaN   
2    GDP per capita, PPP (current international $)  NY.GDP.PCAP.PP.CD   NaN   
3    GDP per capita, PPP (current international $)  NY.GDP.PCAP.PP.CD   NaN   
4    GDP per ca

In [158]:
#Separate table for regions

gdp_per_capita_df = df_list[0]

regions_gdp_df = gdp_per_capita_df.loc[gdp_per_capita_df['Country Name'].isin(["Africa Eastern and Southern",\
                                    "Africa Western and Central",\
                                    "Arab World",\
                                    "Central Europe and the Baltics",\
                                    "Caribbean small states",\
                                    "East Asia & Pacific (excluding high income)",\
                                    "East Asia & Pacific",\
                                    "Europe & Central Asia (excluding high income)",\
                                    "Europe & Central Asia",\
                                    "European Union",\
                                    "Latin America & Caribbean (excluding high income)",\
                                    "Latin America & Caribbean",\
                                    "Middle East & North Africa",\
                                    "Middle East & North Africa (excluding high income)",\
                                    "North America",\
                                    "Sub-Saharan Africa (excluding high income)",\
                                    "Sub-Saharan Africa"])]

regions_gdp_df

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
1,Africa Eastern and Southern,AFE,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3333.520911,3470.550675,3498.125396,3591.099809,3635.564622,3724.868177,3777.972368,3621.058106,3839.470058,NaN
3,Africa Western and Central,AFW,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3950.948889,4143.177054,4075.994459,3998.951876,4045.303263,4160.540965,4264.731035,4174.504565,4409.450961,NaN
7,Arab World,ARB,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,15786.576081,15565.992148,14055.953265,13594.676481,14052.546913,14463.364253,14634.455952,13914.583485,14793.881313,NaN
36,Central Europe and the Baltics,CEB,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,23855.330206,24922.477098,26130.663011,27749.202828,29826.242741,32000.758785,34934.337770,34762.171670,37579.797677,NaN
49,Caribbean small states,CSS,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,15713.674950,15957.916650,15725.925036,15493.716922,16175.340527,16645.404967,17049.114651,15873.377116,17323.612263,NaN
61,East Asia & Pacific (excluding high income),EAP,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,10950.630309,11456.177796,11777.311276,12296.336850,12968.759836,14037.282432,15016.853092,15249.283357,16894.577910,NaN
63,East Asia & Pacific,EAS,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,14181.422297,14693.707141,15108.820636,15609.099672,16329.778648,17427.707753,18369.746587,18556.297577,20249.846275,NaN
64,Europe & Central Asia (excluding high income),ECA,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,19308.593950,19647.580898,19179.959648,19571.836934,20765.977178,22196.511520,22957.574179,22782.038573,24980.915628,NaN
65,Europe & Central Asia,ECS,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,29768.265858,30492.039740,30917.154800,32333.812903,34019.225350,35736.608243,37586.540891,36383.769663,39231.636464,NaN
73,European Union,EUU,"GDP per capita, PPP (current international $)",NY.GDP.PCAP.PP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,36062.688825,37060.689462,38218.564076,40548.841830,42657.741552,44651.979990,47487.022881,45725.190496,48779.382436,NaN


In [159]:
#Regions gdp 2021

regions_gdp_2021_df = regions_gdp_df[['Country Name', 'Country Code', '2021']]
regions_gdp_2021_df.rename(columns = {'Country Name' : 'Region', 'Country Code' : 'Region Code', '2021':'GDP'}, inplace = True)
regions_gdp_2021_df


/var/folders/bz/68gt7jy17hggm4d18zl733y80000gn/T/ipykernel_32409/3861109836.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regions_gdp_2021_df.rename(columns = {'Country Name' : 'Region', 'Country Code' : 'Region Code', '2021':'GDP'}, inplace = True)


,Region,Region Code,GDP
1,Africa Eastern and Southern,AFE,3839.470058
3,Africa Western and Central,AFW,4409.450961
7,Arab World,ARB,14793.881313
36,Central Europe and the Baltics,CEB,37579.797677
49,Caribbean small states,CSS,17323.612263
61,East Asia & Pacific (excluding high income),EAP,16894.577910
63,East Asia & Pacific,EAS,20249.846275
64,Europe & Central Asia (excluding high income),ECA,24980.915628
65,Europe & Central Asia,ECS,39231.636464
73,European Union,EUU,48779.382436


In [160]:
#Same with life expectancy table
life_expectancy_df = df_list[1]
life_expectancy_2021_df = life_expectancy_df[['Country Name', 'Country Code', '2021']]
life_expectancy_2021_df.rename(columns = {'Country Name' : 'Region', 'Country Code' : 'Region Code', '2021':'Life_expectancy'}, inplace = True)
life_expectancy_2021_df

/var/folders/bz/68gt7jy17hggm4d18zl733y80000gn/T/ipykernel_32409/411588204.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  life_expectancy_2021_df.rename(columns = {'Country Name' : 'Region', 'Country Code' : 'Region Code', '2021':'Life_expectancy'}, inplace = True)


,Region,Region Code,Life_expectancy
0,Aruba,ABW,74.626000
1,Africa Eastern and Southern,AFE,62.454585
2,Afghanistan,AFG,61.982000
3,Africa Western and Central,AFW,56.988657
4,Angola,AGO,61.643000
...,...,...,...
261,Kosovo,XKX,76.806000
262,"Yemen, Rep.",YEM,63.753000
263,South Africa,ZAF,62.341000
264,Zambia,ZMB,61.223000


In [161]:
#And literacy table
literacy_df = df_list[2]
literacy_2021_df = literacy_df[['Country Name', 'Country Code', '2021']]
literacy_2021_df.rename(columns = {'Country Name' : 'Region', 'Country Code' : 'Region Code', '2021':'Literacy'}, inplace = True)
literacy_2021_df

/var/folders/bz/68gt7jy17hggm4d18zl733y80000gn/T/ipykernel_32409/279392058.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  literacy_2021_df.rename(columns = {'Country Name' : 'Region', 'Country Code' : 'Region Code', '2021':'Literacy'}, inplace = True)


,Region,Region Code,Literacy
0,Aruba,ABW,NaN
1,Africa Eastern and Southern,AFE,NaN
2,Afghanistan,AFG,37.266041
3,Africa Western and Central,AFW,NaN
4,Angola,AGO,72.279999
...,...,...,...
261,Kosovo,XKX,NaN
262,"Yemen, Rep.",YEM,NaN
263,South Africa,ZAF,NaN
264,Zambia,ZMB,NaN


In [162]:
#Join gdp and life expectancy 2021 tables

gdp_lifeexpectancy_literacy_df = pd.concat([regions_gdp_2021_df, life_expectancy_2021_df], axis=1, join="inner")
gdp_lifeexpectancy_literacy_df2 = gdp_lifeexpectancy_literacy_df.loc[:, ~gdp_lifeexpectancy_literacy_df.columns.duplicated()]
gdp_lifeexpectancy_literacy_df2


,Region,Region Code,GDP,Life_expectancy
1,Africa Eastern and Southern,AFE,3839.470058,62.454585
3,Africa Western and Central,AFW,4409.450961,56.988657
7,Arab World,ARB,14793.881313,70.814483
36,Central Europe and the Baltics,CEB,37579.797677,74.932414
49,Caribbean small states,CSS,17323.612263,70.996259
61,East Asia & Pacific (excluding high income),EAP,16894.577910,75.543291
63,East Asia & Pacific,EAS,20249.846275,76.393934
64,Europe & Central Asia (excluding high income),ECA,24980.915628,71.433174
65,Europe & Central Asia,ECS,39231.636464,76.631526
73,European Union,EUU,48779.382436,80.387131
